# <center> Application of Robust PCA algorithm to background substraction

### Useful libraries

In [1]:
import numpy as np
import time
from scipy.linalg import svd

## Implementation of the algorithm

Below are a few functions useful for the implementation : 

In [2]:
# Shrinkage operator
def shrink_op(M, tau):
    sgn = np.sign(M)
    S = np.abs(M) - tau
    np.place(S,S < 0., 0.)
    return sgn * S

# Threshold operator
# Return also the rank of the output matrix
def thresh_op(M, tau):
    (u, s, v) = svd(M, full_matrices=False)

    s = shrink_op(s, tau)
    rank = np.sum(s > 0.0)
    u, s, v = u[:, :rank], s[:rank], v[:rank, :]
    
    L = np.dot(u, np.dot(np.diag(s), v))
    rank = np.sum(s > 0)
    return L, rank

# Compute the relative distance between two matrix or vector
# ord : same ord as in the numpy function that computes matrix norm
def error(M0,M, norm = 'fro'):
    diff = np.linalg.norm(M0-M,ord = norm)
    if np.count_nonzero(M0) == 0:
        print("Warning : first matrix is a zero matrix")
        return diff
    else:
        return diff/np.linalg.norm(M0, ord = norm)

def missing_data(M, missing_data = False):
    if missing_data:
        missing = ~(np.isfinite(M))
        if np.any(missing):
            print("Warning")
            M = np.array(M)
            M[missing] = 0.0
    else:
        missing = np.zeros_like(M, dtype=bool)
        if not np.all(np.isfinite(M)):
            logging.warn("The matrix has non-finite entries. "
                         "SVD will probably fail.")
#     print("missing = ", np.sum(missing == True))
    return missing

# Update the matrix L, S and Y according to the formulas of the Augmented Lagrange multiplier optimization method
def update_LSY(M,S,Y,mu,lamb):
    L,rank = thresh_op(M - S + Y / mu, 1/ mu)
    S = shrink_op(M - L + Y / mu, lamb / mu)
    Y += mu * (M - L - S)
    return L,S,Y,rank

# Check if the algorithm has converged and print the error ar the end of the computation
def has_converged(i,max_iter,err):
    if i >= max_iter:
        print("Algorithm pcp_ad has not converged at maxiter : ", max_iter)
        print('Error is : ', err)
    else:
        print("Convergence at iteration : ", i)
        print('Error is : ', err)

# Print error, rank of L, sparsity of S and time for each iteration
def print_iter_info(i,err,rank,sparsity, iter_time):
    print(("Iteration {0}: error={1:.3e}, rank={2:d}, sparsity={3:d}, "
           "time={4:.3e}")
          .format(i, err, rank,sparsity , iter_time))

In [3]:
def pcp_ad(M, delta=1e-7, max_iter=500, verbose=False):
    
    """ 
    PCP algorithm solved with Augmented Lagrange Multiplier method
    M : Matrix to be decomposed
    delta : convergence threshold
    Output : L and S
    """
    begin_algo = time.time()
    
    # Initialize all the constants and parameters
    shape = M.shape
    lamb = 1.0 / np.sqrt(np.max(shape))
    mu = 0.25 * np.prod(shape) / np.sum(np.abs(M))
    i = 0
    S = np.zeros(shape)
    Y = np.zeros(shape)
    
    while i < max_iter:
        begin_iter = time.time()
        L,S,Y,rank = update_LSY(M,S,Y,mu,lamb)
        iter_time = time.time() - begin_iter

        err = error(M,L+S)
        if verbose:
            print_iter_info(i,err,rank,np.sum(S > 0),iter_time)
        
        if err < delta:
            break
        else:
            i += 1
            
    has_converged(i,max_iter,err)
    print("Computation time :", time.time() - begin_algo)
    
    return L, S

## Test : generate data for testing the algorithm

### Useful functions to generate the data and check that they respect the hypothesis of the theorem

In [4]:
# Generate a square low rank matrix
# n : dimension
# r : rank
def generate_low_rank(n,r):
    X = np.random.normal(0,1/n,(n,r))
    Y = np.random.normal(0,1/n,(n,r))
    L0 = np.dot(X,Y.transpose())
    return L0

# Generate a square sparse matrix
# n : dimension
# k : number of non zero elements
def generate_sparse(n,k):
    E = np.random.binomial(1,0.5,k)
    np.place(E,E == 0, -1)

    S0 = np.zeros((n,n))
    Omega = []

    while len(Omega) < k:
        i,j = tuple(np.random.randint(0,n,2))
        if (i,j) not in Omega:
            S0[i,j] = E[len(Omega)]
            Omega.append((i,j))
    return S0

# Check that L0 respects the hypothesis of the Theorem : rank, not sparse and incoherence condition
def check_L0_properties(L0,r,k,shape):
    
    (n1, n2) = shape
    print("Rank of L0 = r : ", np.linalg.matrix_rank(L0) == r)
    print('L0 is not sparse : ', np.sum(L0 == 0) < 0.5* n1*n2)
    
    Id_n1, Id_n2 = np.identity(n1), np.identity(n2)
    U,s,V = svd(L0, full_matrices=False)
    U,s,V = U[:,:r], s[:r], V[:r,:]   
    Pu, Pv = np.dot(U,U.transpose()), np.dot(V.transpose(), V)
    cond_U = np.linalg.norm(np.dot(Pu,Id_n1), ord = 2, axis = 1) ** 2
    cond_V = np.linalg.norm(np.dot(Pv,Id_n2), ord = 2, axis = 1) ** 2
    cond_UV = np.max(np.abs(np.dot(U,V))) ** 2
    print( n1 * max(cond_U) / r, n2 * max(cond_V) / r, n1 * n2 * cond_UV / r)

    
# Check that S0 respects the hypothesis of the Theorem : sparsity and not low rank
def check_S0_properties(S0,r,k,shape):
    print("Sparsity of S0 = k :", np.sum(S0 != 0) == k)
    print( "S0 is not a low rank matrix :", np.linalg.matrix_rank(S0) > 0.7 * min(shape))
    
def check_results(L,S, L0,S0):
    print("Error on the low rank matrix : ", error(L0,L))
    print("Error on the sparse matrix : ", error(S0,S))
    print("Rank of original low rank matrix", r)
    print("Rank of the retrieved low rank matrix :", np.linalg.matrix_rank(L))
    print("Number of non zero elements in the original sparse matrix:", k)
    print("Number of non zero elements in the retrieved sparse matrix:", np.sum(S != 0))

### Generating the data and test the algorithm

In [5]:
n = 500
r = int(0.05*n)
k = int(0.05*n*n)

L0 = generate_low_rank(n,r)
S0 = generate_sparse(n,k)

check_L0_properties(L0,r,k,(n,n))
check_S0_properties(S0,r,k,(n,n))

Rank of L0 = r :  True
L0 is not sparse :  True
1.93374366258 1.92833251292 25.2070115947
Sparsity of S0 = k : True
S0 is not a low rank matrix : True


In [6]:
L,S= pcp_ad(L0+S0, max_iter=150, verbose = False)
check_results(L,S, L0,S0)

Algorithm pcp_ad has not converged at maxiter :  150
Error is :  5.11728227739e-05
Computation time : 9.272194862365723
Error on the low rank matrix :  0.609118693483
Error on the sparse matrix :  1.93876191254e-05
Rank of original low rank matrix 25
Rank of the retrieved low rank matrix : 25
Number of non zero elements in the original sparse matrix: 12500
Number of non zero elements in the retrieved sparse matrix: 24740


## Application of the algorithm to a video surveillance : 

In [7]:
from PIL import Image
import glob
import matplotlib.pyplot as plt

In [8]:
# Transform a bitmap sequence into a big matrix where each line correspond to one frame
def bmp_to_array(bitmap_seq):
    data = []
    for bitmap_file in bitmap_seq:
        img = Image.open(bitmap_file).convert("L")
        img_size = img.size
        img = np.array(list(img.getdata()))
        data.append(img)
    return np.array(data), img_size

# Inverse transformation
def array_to_bmp(array,img_size):
    image = Image.new('L', img_size)
    for i in range(len(M)):
        image.putdata(array[i])
        image.save("./data/" + data + "_test_bmp/"+data+"{0:05d}.bmp".format(i))

# Horizontally concatenate a list of images (same image size)
def concatenate_image(list_im,img_size):
    widths, heights = img_size
    total_width = 3 * widths
    image = Image.new('L', (total_width, heights))
    x_offset = 0
    for im in list_im:
        image.paste(im, (x_offset,0))
        x_offset += widths
    return image
    
# Save the images corresponding to M,L,S by concatenating them in one image
def save_results(M,L,S,img_size,data):
    begin = time.time()
    imM = Image.new('L',img_size)
    imL = Image.new('L',img_size)
    imS = Image.new('L',img_size)
    
    for i in range(len(M)):
        imM.putdata(M[i])
        imL.putdata(L[i])
        imS.putdata(S[i])
        image = concatenate_image([imM,imL,imS],img_size)
        image.save("./data/" + data + "_results_bmp/big_picture{0:05d}.bmp".format(i))
    print("Results saved in :", time.time() - begin, 'seconds')

In [9]:
# Choose one dataset
# Available dataset are 'sofa' or 'street'
data = 'sofa'

In [10]:
# To preprocess the video and to convert to a bitmap sequence : 
# Run the following line in a terminal or just decomment
if data == 'sofa':
    !ffmpeg -i ./data/sofa_scaled.mp4  ./data/sofa_bmp/output%04d.bmp
if data == 'street':
    !ffmpeg -i ./data/street_scaled.mp4  ./data/street_bmp/output%04d.bmp

ffmpeg version 3.0.2 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.1.0 (clang-802.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.0.2 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --enable-vda
  libavutil      55. 17.103 / 55. 17.103
  libavcodec     57. 24.102 / 57. 24.102
  libavformat    57. 25.100 / 57. 25.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 31.100 /  6. 31.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  0.100 /  4.  0.100
  libswresample   2.  0.101 /  2.  0.101
  libpostproc    54.  0.100 / 54.  0.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from './data/sofa_scaled.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf56.40.101
  Duratio

In [11]:
# Available choice for data are 'sofa' or 'street'
data = 'sofa'

# Create a list of the pathname of the bitmap sequence
bitmap_seq = np.sort(glob.glob("./data/" + data + "_bmp/*.bmp"))

# Create a matrix containing the frames of the video, one line per frame
M, img_size = bmp_to_array(bitmap_seq)

# Find L and S
L,S = pcp_ad(M, max_iter = 300, verbose = True)

# Save the results as bmp frames
save_results(M,L,S, img_size, data)

Iteration 0: error=4.862e-03, rank=78, sparsity=415130, time=1.925e+00
Iteration 1: error=2.215e-03, rank=81, sparsity=978047, time=1.947e+00
Iteration 2: error=1.587e-03, rank=77, sparsity=1513785, time=1.824e+00
Iteration 3: error=1.244e-03, rank=72, sparsity=1978627, time=1.851e+00
Iteration 4: error=1.058e-03, rank=68, sparsity=2289802, time=1.782e+00
Iteration 5: error=9.534e-04, rank=64, sparsity=2508036, time=1.768e+00
Iteration 6: error=8.922e-04, rank=63, sparsity=2675111, time=1.728e+00
Iteration 7: error=8.365e-04, rank=61, sparsity=2805481, time=1.732e+00
Iteration 8: error=7.784e-04, rank=61, sparsity=2911079, time=1.749e+00
Iteration 9: error=7.183e-04, rank=60, sparsity=2996926, time=1.824e+00
Iteration 10: error=6.587e-04, rank=60, sparsity=3070918, time=1.775e+00
Iteration 11: error=6.100e-04, rank=59, sparsity=3136218, time=1.723e+00
Iteration 12: error=5.685e-04, rank=60, sparsity=3191891, time=1.732e+00
Iteration 13: error=5.324e-04, rank=61, sparsity=3238073, time=

In [12]:
# Create a video to based on the matrix obtained with the algorithm
if data == 'sofa':
    !ffmpeg -i ./data/sofa_results_bmp/big_picture%05d.bmp ./data/sofa_results_vid.avi
if data == 'street':
    !ffmpeg -i ./data/street_results_bmp/big_picture%05d.bmp ./data/street_results_vid.avi

ffmpeg version 3.0.2 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.1.0 (clang-802.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.0.2 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --enable-vda
  libavutil      55. 17.103 / 55. 17.103
  libavcodec     57. 24.102 / 57. 24.102
  libavformat    57. 25.100 / 57. 25.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 31.100 /  6. 31.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  0.100 /  4.  0.100
  libswresample   2.  0.101 /  2.  0.101
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from './data/sofa_results_bmp/big_picture%05d.bmp':
  Duration: 00:00:07.56, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: bmp, pal8, 960x240, 25 fps, 25 tbr, 25 tbn, 25 tbc
Output #0, avi, 